In [ ]:
# Importamos y configuramos la sesión de Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Contribuciones2016-2018") \
    .getOrCreate()

# Mostrar configuración básica
spark


In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, DoubleType

# Esquema para PADRON_COMPLETO.csv
schema_padron = StructType([
    StructField("CEDULA", StringType(), nullable=False),
    StructField("CODELEC", StringType(), nullable=False),
    StructField("RELLENO", StringType(), nullable=True),
    StructField("FECHACADUC", StringType(), nullable=True),
    StructField("JUNTA", StringType(), nullable=True),
    StructField("NOMBRE", StringType(), nullable=True),
    StructField("1_APELLIDO", StringType(), nullable=True),
    StructField("2_APELLIDO", StringType(), nullable=True)
])

# Esquema para distelec.csv
schema_distr = StructType([
    StructField("CODELE", StringType(), nullable=False),
    StructField("PROVINCIA", StringType(), nullable=True),
    StructField("CANTON", StringType(), nullable=True),
    StructField("DISTRITO", StringType(), nullable=True)
])

# Leemos PADRON_COMPLETO
padron_df = spark.read.csv("/FileStore/tables/PADRON_COMPLETO.csv",
                           sep=",", header=True,
                           schema=schema_padron,
                           dateFormat="yyyyMMdd")

# Leemos distelec
distr_df = spark.read.csv("/FileStore/tables/distelec.csv",
                          sep=",", header=True,
                          schema=schema_distr)

# Leemos contribuciones, sin encabezado útil, saltamos primeras 3 líneas
raw_contrib = spark.read.option("header", True) \
    .option("inferSchema", False) \
    .csv("/FileStore/tables/por-ciclo-2016-2018.csv", sep=",", header=False)

# Quitamos líneas de título manualmente
contrib = raw_contrib.filter(raw_contrib._c0.isNull() == False) \
    .filter(raw_contrib._c0 != "TRIBUNAL SUPREMO DE ELECCIONES")


In [ ]:
from pyspark.sql.functions import col, to_date, regexp_replace, udf
from pyspark.sql.types import DoubleType

# Renombramos columnas relevantes y seleccionamos las necesarias
contrib2 = contrib.select(
    col("_c1").alias("TIPO_CONTRIBUCION"),
    col("_c2").alias("TIPO_PERSONA"),
    col("_c3").alias("ESCALA"),
    col("_c4").alias("PARTIDO"),
    col("_c5").alias("FECHA_STR"),
    col("_c6").alias("CEDULA"),
    col("_c7").alias("NOMBRE_CONTRIBUYENTE"),
    col("_c8").alias("MONTO_STR")
)

# Convertimos FECHA a DateType (formato MM/dd/yy)
contrib3 = contrib2.withColumn("FECHA",
    to_date(col("FECHA_STR"), "MM/dd/yy")) \
  .drop("FECHA_STR")

# UDF para limpiar monto y convertir a Double
@udf(returnType=DoubleType())
def parse_monto(s):
    if s is None: return None
    return float(s.replace('"','').replace(',',''))

contrib4 = contrib3.withColumn("MONTO",
    parse_monto(col("MONTO_STR"))) \
  .drop("MONTO_STR")

# Cacheamos para reutilizar
contrib4.cache()


In [ ]:
# Primero, mapeamos el padron para extraer CODELEC de 6 dígitos
padron2 = padron_df.withColumn("CODELE", col("CODELEC"))

# Join padron ↔ distritos para tener PROVINCIA/CANTON/DISTRITO en las contribuciones
contrib_geo = contrib4.join(
    padron2.select("CEDULA","CODELE"),
    on="CEDULA", how="left"
).join(
    distr_df, on="CODELE", how="left"
).cache()

# Vemos esquema final
contrib_geo.printSchema()


In [ ]:
from pyspark.sql.functions import countDistinct

# Contamos CEDULA distinta por partido
cont_por_partido = contrib_geo.groupBy("PARTIDO") \
    .agg(countDistinct("CEDULA").alias("NUM_CONTRIBUYENTES")) \
    .orderBy(col("NUM_CONTRIBUYENTES").desc())

cont_por_partido.show(truncate=False)


In [ ]:
# Filtramos provincia San José y agregamos suma de MONTO
dist_sanjose = contrib_geo.filter(col("PROVINCIA") == "SAN JOSE") \
    .groupBy("CANTON") \
    .agg(
      countDistinct("CEDULA").alias("NUM_CONTRIBUYENTES"),
      # solo efectivo y especie juntas
      expr("sum(MONTO) as MONTO_TOTAL")
    ).orderBy(col("MONTO_TOTAL").desc())

dist_sanjose.show(truncate=False)


In [ ]:
# Separamos por tipo de contribución
pivot_tipo = contrib_geo.groupBy("CANTON") \
    .pivot("TIPO_CONTRIBUCION", ["EFECTIVO","ESPECIE"]) \
    .count().na.fill(0) \
    .orderBy(col("EFECTIVO").desc())

pivot_tipo.show(truncate=False)


In [ ]:
**Particiones y cache**  
- `contrib_geo` está cached para acelerar múltiples lecturas.  
- Podemos particionar por `CANTON` si vamos a escribir resultados a disco:

```python
contrib_geo.repartition("CANTON") \
    .write.mode("overwrite") \
    .parquet("/mnt/data/output/contrib_by_canton")
